### Imports

In [ ]:
import main_var
mv = main_var.main_var()

from scrap_lib import *
from pars_lib import *
from openai_module_lib import *
from embedding_keyword_module_lib import *
from visualization_module_lib import *
from dimension_reduc_lib import *
from agg_lib import *
from utils_art import *

In [ ]:
_ENTRY_NUMBER = 1000000
_STEP_PCT = 0.01

# Querring, Scraping and saving articles
### Querring of articles `(1_1_query)`

In [ ]:
from scrap_lib import *
# df_query = openDFcsv(mv.query_path,mv.query_filename)
df_query = loop_scraping(number_topics=9,
                   startDate='2010-01-01',
                   endDate='2024-01-01',
                   sampling_1=14,
                   sampling_2=20,
                   save_steps=True,
                   save_final=True,
                   display=True)

##### Display Statistics

In [ ]:
plotDFstatisticsQuerry(df_query,100,False)
displayStats(df_query)

##### Filter articles with a source that is too/not enought scrapped and source/category/date

In [ ]:
# df = openDFcsv(mv.query_path,mv.query_filename)
categroy_l = ["WORLD","NATION","BUSINESS","TECHNOLOGY"]
source_l = ["Newsweek","Yahoo Finance","ESPN","Fox News","World Health Organization (WHO)","Pew Research Center","The New York Times","Nature.com","Tax Foundation","Union of Concerned Scientists"]

df_query_filter = filterQuerryDataset(thd_high=100000,thd_low=110,display_stats=True,display_end_stats=True,save=True)
# plotDFstatisticsQuerry(df_query_filter)
# displayStats(df_query_filter)

df_query_filter = selectOnDf(df_query_filter,date_start="2010-01-01", date_end="2024-01-01", categroy_list=categroy_l, source_list=source_l)
# plotDFstatisticsQuerry(df_query_filter)
# displayStats(df_query_filter)
saveDFcsv(df_query_filter,mv.query_path,"query_file_filtered")

### Scrap articles from the web based on the query file `(1_2_scarp & 1_3_article)`

In [ ]:
from pars_lib import *
# df_scrap = openDFcsv(mv.scarp_path,mv.scarp_filename) ##(1) 20995  (2) 26860  (3) 18283  (sum all valid 59905) last hash 69843
df_scrap = readArticleFileTable(index_from=133454,# 25088  133454
                          index_to=_ENTRY_NUMBER,
                          save_articles=True,
                          save_final=True,
                          save_steps=True,
                          display_data=True,
                          step_pct=_STEP_PCT,
                          add_nlp=2,
                          filtered_input_df=True)

### Join the two results `(1_4_join)`

In [ ]:
from agg_lib import *
# df_join1 = openDFcsv(mv.join1_path,mv.join1_filename)
df_join1 = joinQuerryAndParse(save=True,
                        remove_invalid=True,
                        display=True,
                        filtered_input_df=True,
                        union_df=None)

# Generation of Embedding and Keyword data
### Embedding with OpenAI API `(2_1_embdedding)`

In [ ]:
from openai_module_lib import *
# df_embdedding = openDFcsv(mv.embdedding_path,mv.embdedding_filename)
df_embdedding = mainGeneration(articleTRUEquestionFALSE=True,
                     completionTRUEembeddingFALSE=False,
                     dimension=100,
                     max_prompt=_ENTRY_NUMBER,
                     token_max_emb=7500,
                     cara_max_emb=10000,
                     save_final=True,
                     display_df_var=True,
                     save_steps=True,
                     step_pct=_STEP_PCT)

##### Extract embedding from file `(2_1_embdedding_raw)`

In [ ]:
from embedding_keyword_module_lib import *
emb_mat = extractEmbeddingFromFile(_ENTRY_NUMBER,True)

##### Use dime,tion reduction on emneded data

In [2]:
from dimension_reduc_lib import *
df_embdedding2 = generateDimReducedDF(n_components=3,
                                          norm_output=True,
                                      active_sel=[True,True,True,False])

Input dataframe shape : (121200, 8)


### Keyword extraction and NLP analysis `(2_2_keyword)`

In [ ]:
from embedding_keyword_module_lib import *
# df = openDFcsv(mv.keyword_path,mv.keyword_filename)
df_keyword = mainKeywordWF(entry_limit=_ENTRY_NUMBER,
              common_word_max=3000,
              add_nlp_stats=True,
              nlp_source_col="word_combined_all")

# Final join and visualization
### Joining Quering/Scraping/Embedding/Keywords `(3_1_join)`

In [3]:
from agg_lib import *
# df_join2 = openDFcsv(mv.join2_path,mv.join2_filename)
joinAllDF(save=True,
         union_df=None)

IMPORT : agg_lib
JOIN1 dataset loaded from  C:/Users/User/OneDrive/Desktop/Article_LLM/main_files/1_4_join_main/.main/
JOIN1 dataset has entry length of : 121531 

EMBEDDING dataset loaded from  C:/Users/User/OneDrive/Desktop/Article_LLM/main_files/2_1_embdedding_main/.main/
EMBEDDING dataset has entry length of : 121200  (99.73% of querry data)

KEYWORD dataset loaded from  C:/Users/User/OneDrive/Desktop/Article_LLM/main_files/2_2_keyword_main/.main/
KEYWORD dataset has entry length of : 121531  (100.0% of querry data)

TOTAL yield : from 121531  to  111668 (91.88% yeald)



,title_quer,link,published,source_url,source_title,category,year,year_month,pk,url_list,...,vs.neg_k,vs.comp_k,ts.neg_k,ts.neu_k,ts.pos_k,tb.class,vs.class,ts.class,al.pos_k,al.neg_k
hash_key,,,,,,,,,,,,,,,,,,,,,
cec0db5c203d47d86e7ab53214cc4e00e275f7fe,'Top Chef' secrets revealed: From the judges' ...,https://news.google.com/rss/articles/CBMiYGh0d...,2010-01-12,https://ew.com,Entertainment Weekly News,TOP,2010,2010-01,CBMiYGh0dHBzOi8vZXcuY29tL2FydGljbGUvMjAxMC8wMS...,['ew'],...,0.000,0.5574,0.025946,0.893159,0.080895,True,True,True,1.000000,0.000000
6b92fd07707bdadb8b4ae2c0564cadd585fe8f34,Academy chosen among nation's top 100 best-val...,https://news.google.com/rss/articles/CBMicGh0d...,2010-01-13,https://www.af.mil,Air Force Link,TOP,2010,2010-01,CBMicGh0dHBzOi8vd3d3LmFmLm1pbC9OZXdzL0FydGljbG...,['af'],...,0.000,0.9118,0.012387,0.622896,0.364717,True,True,True,1.000000,0.000000
ed439993352c671f6cfe0df72613ea80daa45909,Top of the Pops - The New Yorker,https://news.google.com/rss/articles/CBMiPWh0d...,2010-01-03,https://www.newyorker.com,The New Yorker,TOP,2010,2010-01,CBMiPWh0dHBzOi8vd3d3Lm5ld3lvcmtlci5jb20vbWFnYX...,['newyorker'],...,0.000,0.2023,0.011010,0.736936,0.252054,True,True,True,1.000000,0.000000
509a4d1d83b8a1e48eeb7b114028f7ac475d1d9f,Top 10 Ways to Fix JRPGs - IGN,https://news.google.com/rss/articles/CBMiQGh0d...,2010-01-11,https://www.ign.com,IGN,TOP,2010,2010-01,CBMiQGh0dHBzOi8vd3d3Lmlnbi5jb20vYXJ0aWNsZXMvMj...,['ign'],...,0.000,0.2023,0.126121,0.793061,0.080818,True,True,False,0.666667,0.333333
4cbce276500e85b4e25d3b81b353ac3479b3e990,Carter's Top 50 NFL Wide Receivers Of The 2009...,https://news.google.com/rss/articles/CBMiX2h0d...,2010-01-14,https://bleacherreport.com,Bleacher Report,TOP,2010,2010-01,CBMiX2h0dHBzOi8vYmxlYWNoZXJyZXBvcnQuY29tL2FydG...,['bleacherreport'],...,0.000,0.7096,0.003748,0.046944,0.949309,True,True,True,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ac47756187d8204c1069ed19d0eaaaee5ecbdcfd,Protecting against cognitive decline - Harvard...,https://news.google.com/rss/articles/CBMiUWh0d...,2023-12-07,https://www.health.harvard.edu,Harvard Health,HEALTH,2023,2023-12,CBMiUWh0dHBzOi8vd3d3LmhlYWx0aC5oYXJ2YXJkLmVkdS...,"['health', 'harvard']",...,0.000,0.0000,0.018181,0.849152,0.132667,False,False,True,0.333333,0.666667
56219df0d13d4b1c30660ded333858b7f79006d3,School of Health Sciences hosts annual researc...,https://news.google.com/rss/articles/CBMiXWh0d...,2023-12-13,https://www.elon.edu,Today at Elon,HEALTH,2023,2023-12,CBMiXWh0dHBzOi8vd3d3LmVsb24uZWR1L3UvbmV3cy8yMD...,['elon'],...,0.000,0.6597,0.009514,0.945206,0.045280,True,True,True,1.000000,0.000000
13ffebe1b134d6f328951d0acddc3cc5c84a30cd,WHO and UNICEF publish new psychological inter...,https://news.google.com/rss/articles/CBMiigFod...,2023-12-20,https://www.who.int,World Health Organization (WHO),HEALTH,2023,2023-12,CBMiigFodHRwczovL3d3dy53aG8uaW50L25ld3MvaXRlbS...,['who'],...,0.089,0.5423,0.008702,0.819572,0.171727,True,True,True,1.000000,0.000000


,title_quer,link,published,source_url,source_title,category,year,year_month,pk,url_list,...,vs.neg_k,vs.comp_k,ts.neg_k,ts.neu_k,ts.pos_k,tb.class,vs.class,ts.class,al.pos_k,al.neg_k
hash_key,,,,,,,,,,,,,,,,,,,,,
cec0db5c203d47d86e7ab53214cc4e00e275f7fe,'Top Chef' secrets revealed: From the judges' ...,https://news.google.com/rss/articles/CBMiYGh0d...,2010-01-12,https://ew.com,Entertainment Weekly News,TOP,2010,2010-01,CBMiYGh0dHBzOi8vZXcuY29tL2FydGljbGUvMjAxMC8wMS...,['ew'],...,0.000,0.5574,0.025946,0.893159,0.080895,True,True,True,1.000000,0.000000
6b92fd07707bdadb8b4ae2c0564cadd585fe8f34,Academy chosen among nation's top 100 best-val...,https://news.google.com/rss/articles/CBMicGh0d...,2010-01-13,https://www.af.mil,Air Force Link,TOP,2010,2010-01,CBMicGh0dHBzOi8vd3d3LmFmLm1pbC9OZXdzL0FydGljbG...,['af'],...,0.000,0.9118,0.012387,0.622896,0.364717,True,True,True,1.000000,0.000000
ed439993352c671f6cfe0df72613ea80daa45909,Top of the Pops - The New Yorker,https://news.google.com/rss/articles/CBMiPWh0d...,2010-01-03,https://www.newyorker.com,The New Yorker,TOP,2010,2010-01,CBMiPWh0dHBzOi8vd3d3Lm5ld3lvcmtlci5jb20vbWFnYX...,['newyorker'],...,0.000,0.2023,0.011010,0.736936,0.252054,True,True,True,1.000000,0.000000
509a4d1d83b8a1e48eeb7b114028f7ac475d1d9f,Top 10 Ways to Fix JRPGs - IGN,https://news.google.com/rss/articles/CBMiQGh0d...,2010-01-11,https://www.ign.com,IGN,TOP,2010,2010-01,CBMiQGh0dHBzOi8vd3d3Lmlnbi5jb20vYXJ0aWNsZXMvMj...,['ign'],...,0.000,0.2023,0.126121,0.793061,0.080818,True,True,False,0.666667,0.333333
4cbce276500e85b4e25d3b81b353ac3479b3e990,Carter's Top 50 NFL Wide Receivers Of The 2009...,https://news.google.com/rss/articles/CBMiX2h0d...,2010-01-14,https://bleacherreport.com,Bleacher Report,TOP,2010,2010-01,CBMiX2h0dHBzOi8vYmxlYWNoZXJyZXBvcnQuY29tL2FydG...,['bleacherreport'],...,0.000,0.7096,0.003748,0.046944,0.949309,True,True,True,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ac47756187d8204c1069ed19d0eaaaee5ecbdcfd,Protecting against cognitive decline - Harvard...,https://news.google.com/rss/articles/CBMiUWh0d...,2023-12-07,https://www.health.harvard.edu,Harvard Health,HEALTH,2023,2023-12,CBMiUWh0dHBzOi8vd3d3LmhlYWx0aC5oYXJ2YXJkLmVkdS...,"['health', 'harvard']",...,0.000,0.0000,0.018181,0.849152,0.132667,False,False,True,0.333333,0.666667
56219df0d13d4b1c30660ded333858b7f79006d3,School of Health Sciences hosts annual researc...,https://news.google.com/rss/articles/CBMiXWh0d...,2023-12-13,https://www.elon.edu,Today at Elon,HEALTH,2023,2023-12,CBMiXWh0dHBzOi8vd3d3LmVsb24uZWR1L3UvbmV3cy8yMD...,['elon'],...,0.000,0.6597,0.009514,0.945206,0.045280,True,True,True,1.000000,0.000000
13ffebe1b134d6f328951d0acddc3cc5c84a30cd,WHO and UNICEF publish new psychological inter...,https://news.google.com/rss/articles/CBMiigFod...,2023-12-20,https://www.who.int,World Health Organization (WHO),HEALTH,2023,2023-12,CBMiigFodHRwczovL3d3dy53aG8uaW50L25ld3MvaXRlbS...,['who'],...,0.089,0.5423,0.008702,0.819572,0.171727,True,True,True,1.000000,0.000000


### Visualisation `(3_2_visu)`

In [ ]:
df_main = openDFcsv(mv.join2_path,mv.join2_filename)
out_list, out_list_label = selectAlgoDfList(df_main,3, norm_output=True, active_sel=[True,False,False,False])
_2dList_out, _3dList_out = getRenderLists()
for i in range(len(out_list)):
    plt_2dList = renderAllOptions(out_list[i],True,_2dList_out,out_list_label[i])
    savePlotList(plt_2dList,mv.visu_path,"2D_"+out_list_label[i])
    plt_3dList = renderAllOptions(out_list[i],False,_3dList_out,out_list_label[i])
    savePlotList(plt_3dList,mv.visu_path,"3D_"+out_list_label[i])

In [ ]:
# df_join1 = openDFcsv(mv.join1_path,mv.join1_filename)
# df_embdedding = openDFcsv(mv.embdedding_path,mv.embdedding_filename)
# df = openDFcsv(mv.keyword_path,mv.keyword_filename)

# df = openDFcsv(mv.join2_path,mv.join2_filename)
#df_gen = generateDimReducedDF(3,True,[True,True,True,True])


In [ ]:
df = openDFcsv(mv.join2_path,mv.join2_filename)
val1 = len(list(df.keys()))
print(val1)
rename_dict = {"hash_key":"Primary Key", "title_quer":"Main Title","link":"Google Link","published":"Date Publication","source_url":"Real Link","source_title":"Name of the source","category":"Main Category","year":"Year","year_month":"Year & Month", "url_TLD":"url_extention","keywords_list":"List Of Keywords","tb.sent":"Number of sentences","tb.noun":"Number of Nouns","tb.word":"Number of Words","tb.char":"Number of Character","tb.pol":"Polarity","tb.sub":"Subjectivity","tb.polaj":"Polatity beetween 1-0","tb.pos":"Positivity_1","tb.neg":"Negativity_1","vs.pos":"Positivity_2","vs.neg":"Negativity_2","ts.pos":"Positivity_3","ts.neg":"Negativity_3","al.pos":"Positivity_all","al.neg":"Negativity_all"}
val2 = len(list(rename_dict.keys()))
print(val2)
# print(len(list(rename_dict.keys())))
# print(len(list(rename_dict.values())))
df = openDFcsv(mv.join2_path,mv.join2_filename)
df.rename(columns=rename_dict, inplace=True)
val3 = len(list(df.keys()))
print(val3)

In [ ]:
col_list = ["category","source_title","year_month","url_TLD","year"]
df_list_len = calculateStatsColList(df,col_list,"len",display_df=True)
df_list_npl = calculateStatsColList(df,col_list,"nlp",display_df=True,display_stats=False) 

In [ ]:
# df = openDFcsv(mv.join1_path,mv.join1_filename)
# genrateKeywordExtract(df, 9641,500,"keywords_list")
#df = mainKhttp://localhost:8888/notebooks/OneDrive/Desktop/Article_LLM/main_codebase/MAIN_CODE.ipynb#eywordWF(9641,1000)

In [ ]:
# df = openDFcsv(mv.query_path,mv.query_filename)
# df3 = selectOnDf(df2,"","")
# plotDFstatisticsQuerry(df,70)
# displayStats(df)
# saveDFcsv(df3,mv.query_path,mv.query_filename)



# df = loop_scraping(9,'1999-03-27','2024-03-19',sampling_1=5,sampling_2=5,save_steps=True,save_final=True,display=True)
# plotDFstatisticsQuerry(df,70)
# displayStats(df)

# df = openDFcsv(mv.query_path,mv.query_filename)
# df2 =  filterQuerryDataset(df,150,10,True,True,True)
# plotDFstatisticsQuerry(df2,70,True)
# displayStats(df2)

# df = openDFcsv(mv.query_path,mv.query_filename)
df_querry = readArticleFileTable(index_from=0,index_to=200,save_articles=False,save_final=True,save_steps=True,display_df=True,step_pct=0.01,add_nlp=2,filtered_input_df=True)
# df_join = joinQuerryAndParse(save=True,remove_invalid=True,display=True,filtered_input_df=True)# df_embd = mainGeneration(True,False,100,99999999,7500,9999,True,True,True,0.005)
# emb_mat = extractEmbeddingFromFile(9999999999)
# df_key = mainKeywordWF(99999999999999,1000)
# df_final = mainJoinOut()

In [ ]:
# df = openDFcsv(mv.query_path,mv.query_filename)
# df3 = selectOnDf(df2,"","")
# plotDFstatisticsQuerry(df,70)
# displayStats(df)
# saveDFcsv(df3,mv.query_path,mv.query_filename)


max_num = 10000000
step_pct=0.005
df = loop_scraping(9,'2000-01-01','2024-01-01',sampling_1=5,sampling_2=5,save_steps=True,save_final=True,display=True)
df2 =  filterQuerryDataset(df,150,10,True,True,True)
df_querry = readArticleFileTable(index_from=0,index_to=max_num,save_articles=True,save_final=True,save_steps=True,display_df=True,step_pct=step_pct,add_nlp=2,filtered_input_df=True)
df_join = joinQuerryAndParse(save=True,remove_invalid=True,display=True,filtered_input_df=True)
df_embd = mainGeneration(True,False,10,max_num,7500,1000,True,True,True,step_pct)
emb_mat = extractEmbeddingFromFile(max_num)
df_key = mainKeywordWF(max_num,100)
df_final = mainJoinOut()

In [ ]:
df = loop_scraping(9,'2000-01-01','2024-01-01',sampling_1=1,sampling_2=1,save_steps=True,save_final=True,display=True)

In [ ]:
df = openDFcsv(mv.join1_path,mv.join1_filename)
# col_list = ["category"] #,"url_TLD" #,"source_title","year_month",
# col_list = ["published"]
col_list = ["category","year","source_title","year_month"]#
df_list1 = calculateStatsColList(df,col_list,"len",display_df=True)
df_list2 = calculateStatsColList(df,col_list,"nlp",display_df=True,display_stats=True) #,out_raw=False

In [ ]:
import matplotlib.pyplot as plt
for i in range(len(col_list)) :
    col_agg = col_list[i]
    table = df_list2[i]
    table.plot(kind='bar')
    plot = df_list[2].plot.barh(y='polarity_per_count',use_index=True, legend=True, fontsize= 8, title="test") #, figsize=(5, 5) #,ax=axes[0],ay=axes[0], ylabel=""


In [ ]:
# fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(5, 5))
# df_list[0].ix[5].plot(kind='bar'); plt.axhline(0, color='k')
    

In [ ]:
import plotly.express as px

fig = px.line(df_list2[1], x=df_list2[1].index, y="count") #, color='country'
fig.show()
fig = px.line(df_list2[1], x=df_list2[1].index, y="polarity_per_count") #, color='country'
fig.show()
fig = px.line(df_list2[1], x=df_list2[1].index, y="positivity_per_count") #, color='country'
fig.show()
fig = px.line(df_list2[1], x=df_list2[1].index, y="negativity_per_count") #, color='country'
fig.show()

In [ ]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

app = Dash(__name__)


app.layout = html.Div([
    html.H4('Life expentancy progression of countries per continents'),
    dcc.Graph(id="graph"),
    dcc.Checklist(
        id="checklist",
        options=["Asia", "Europe", "Africa","Americas","Oceania"],
        value=["Americas", "Oceania"],
        inline=True
    ),
])


@app.callback(
    Output("graph", "figure"), 
    Input("checklist", "value"))
def update_line_chart(continents):
    df = px.data.gapminder() # replace with your own data source
    mask = df.continent.isin(continents)
    fig = px.line(df[mask], 
        x="year", y="lifeExp", color='country')
    return fig


app.run_server(debug=True)

In [ ]:
d1 = {"d1":1}
d2 = {"d2":2}
d3 = d1|d2
print(d3)